In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

# Extra-Pのオプション
modelerNames: list[str] = [
    # "refining", 
    "multi-parameter",
    "default --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1", 
    # "basic --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1"
    ]

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]
list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

/workspace


DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


In [2]:
# LULESH

In [3]:
expVar: list[str] = ["process", "iteration", "size"]
resVar: str = "Exclusive"

trainDF: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

testDF: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

testDF = testDF.reset_index()
trainDF = trainDF.reset_index()

testDF["functionName"] = testDF["Name"]
trainDF["functionName"] = trainDF["Name"]

functionNames: list[str] = sorted(list(set(trainDF["Name"])))

dict_symbols = {}
for elem in expVar:
    dict_symbols[elem] = symbols(elem, real=True)

target_env = [
    (dict_symbols["size"], test_lulesh_sizes[-1]),
    (dict_symbols["iteration"], test_lulesh_iterations[-1]),
    (dict_symbols["process"], test_lulesh_processes[-1]),
]

In [4]:
# Extra-P による関数コール回数予測

resVar: str = "#Call"
expVar: list[str] = ["process", "iteration", "size"]

dict_models_perFunc: dict[str, dict[str, str]] = {}

for functionName in functionNames:

    trainDF_perFunc: pd.DataFrame = trainDF[
        trainDF["Name"] == functionName
    ].reset_index()

    str_ExtraPinputData_call: str = gen_ExtraPinputDataFromDF(
        inputDF=trainDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )

    # モデル格納用の辞書
    dictModels: dict[str, str] = {}

    # Extra-P によるモデルの構築
    converted_functionName: str = re.sub(
        r'[\\|/|:|?|.|"|<|>|\|\(|\)|]', "-", functionName
    )
    fileName: str = f"input_lulesh_@{converted_functionName}"
    fileDir: str = f"./extra-p_docker/share/"
    filePath: str = fileDir + fileName
    print(f"functionName = {functionName}")
    print(f"converted_functionName = {converted_functionName}")
    print(f"filePath = {filePath}")
    with open(file=filePath, mode="w") as f:
        f.write(str_ExtraPinputData_call)

    for modelerName in modelerNames:
        str_resFromExtraP: str = subprocess.run(
            f"extrap --text {filePath} --modeler {modelerName}  | grep Model",
            stdout=subprocess.PIPE,
            text=True,
            shell=True,
        ).stdout
        str_resFromExtraP = str_resFromExtraP.replace("Model: ", "")
        str_resFromExtraP = convert_log(str_resFromExtraP)
        model_fromExtraP = sympify(str_resFromExtraP, locals=dict_symbols)
        dictModels[modelerName] = model_fromExtraP

    # 提案手法によるモデルの構築
    bestModelDict: dict = return_bestModelObject(
        inputDF=trainDF_perFunc,
        list_expVar=expVar,
        list_resVar=[resVar],
        list_modelName=list_modelName,
    )

    dictModels["proposal"] = bestModelDict["object"]

    dict_models_perFunc[functionName] = dictModels

functionName = .TAU_application
converted_functionName = -TAU_application
filePath = ./extra-p_docker/share/input_lulesh_@-TAU_application


Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/sit

functionName = MPI_Allreduce()
converted_functionName = MPI_Allreduce--
filePath = ./extra-p_docker/share/input_lulesh_@MPI_Allreduce--


Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]


functionName = MPI_Barrier()
converted_functionName = MPI_Barrier--
filePath = ./extra-p_docker/share/input_lulesh_@MPI_Barrier--


...

Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/sit

functionName = void_ParseCommandLineOptions(int_char_**_Int_t_cmdLineOpts
converted_functionName = void_ParseCommandLineOptions-int_char_**_Int_t_cmdLineOpts
filePath = ./extra-p_docker/share/input_lulesh_@void_ParseCommandLineOptions-int_char_**_Int_t_cmdLineOpts


Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/sit

functionName = void_VerifyAndWriteFinalOutput(Real_t_Domain
converted_functionName = void_VerifyAndWriteFinalOutput-Real_t_Domain
filePath = ./extra-p_docker/share/input_lulesh_@void_VerifyAndWriteFinalOutput-Real_t_Domain


Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]


In [5]:
_list_series: list[pd.Series] = []

for functionName in functionNames:
    pass
    testDF_perFunc: pd.DataFrame = testDF[testDF["Name"] == functionName].reset_index()

    tmp_series: pd.Series = pd.Series(dtype="object")
    # Extra-P による予測
    for modelerName in modelerNames:
        tmp_series[modelerName] = (
            dict_models_perFunc[functionName][modelerName].subs(target_env).evalf()
        )
    # 提案手法による予測
    _tmp_testDF_perFunc: pd.DataFrame = testDF_perFunc[
        (testDF_perFunc["process"] == test_lulesh_processes[-1])
        & (testDF_perFunc["iteration"] == test_lulesh_iterations[-1])
        & (testDF_perFunc["size"] == test_lulesh_sizes[-1])
        & (testDF_perFunc["Name"] == functionName)
    ]
    try:
        tmp_series["proposal"] = float(
            np.array(
                dict_models_perFunc[functionName]["proposal"].predict(
                    inputDF=_tmp_testDF_perFunc[expVar]
                )
            )
        )
    except:
        warnings.warn(f"{functionName} has no data at testDF")
        continue
    # 実測値
    tmp_series["real_call"] = _tmp_testDF_perFunc.reset_index(drop=True).loc[0]["#Call"]
    tmp_series["functionName"] = functionName

    _list_series.append(tmp_series)

In [6]:
DF_about_call_at_lulesh: pd.DataFrame = pd.DataFrame(_list_series)
DF_about_call_at_lulesh = add_relativeErrorRateCol(
    inputDF=DF_about_call_at_lulesh,
    real_colName="real_call",
    predicted_colName="default --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1",
    targetColName="real_call vs default",
)
# DF_about_call_at_lulesh = add_relativeErrorRateCol(
#     inputDF=DF_about_call_at_lulesh,
#     real_colName="real_call",
#     predicted_colName="basic",
#     targetColName="real_call vs basic",
# )
# DF_about_call_at_lulesh = add_relativeErrorRateCol(
#     inputDF=DF_about_call_at_lulesh,
#     real_colName="real_call",
#     predicted_colName="refining",
#     targetColName="real_call vs refining",
# )
DF_about_call_at_lulesh = add_relativeErrorRateCol(
    inputDF=DF_about_call_at_lulesh,
    real_colName="real_call",
    predicted_colName="multi-parameter",
    targetColName="real_call vs multi-parameter",
)
DF_about_call_at_lulesh = add_relativeErrorRateCol(
    inputDF=DF_about_call_at_lulesh,
    real_colName="real_call",
    predicted_colName="proposal",
    targetColName="real_call vs proposal",
)
DF_about_call_at_lulesh

,multi-parameter,"default --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1",proposal,real_call,functionName,real_call vs default,real_call vs multi-parameter,real_call vs proposal
0,0.999999999999999,0.999999999999999,1.000000e+00,1.000000e+00,.TAU_application,1.11022302462516e-13,1.11022302462516e-13,0.000000e+00
1,1023.00000000000,1023.00000000000,1.023000e+03,1.023000e+03,MPI_Allreduce(),2.22261657324763e-14,2.22261657324763e-14,0.000000e+00
2,0.999999999999999,0.999999999999999,1.000000e+00,1.000000e+00,MPI_Barrier(),1.11022302462516e-13,1.11022302462516e-13,0.000000e+00
3,9221.00000000000,9221.00000000000,9.221000e+03,9.221000e+03,MPI_Comm_rank(),1.97265958523572e-14,1.97265958523572e-14,3.945319e-14
4,0.999999999999999,0.999999999999999,1.000000e+00,1.000000e+00,MPI_Comm_size(),1.11022302462516e-13,1.11022302462516e-13,0.000000e+00
5,0.999999999999999,0.999999999999999,1.000000e+00,1.000000e+00,MPI_Finalize(),1.11022302462516e-13,1.11022302462516e-13,0.000000e+00
6,0.999999999999999,0.999999999999999,1.000000e+00,1.000000e+00,MPI_Init(),1.11022302462516e-13,1.11022302462516e-13,0.000000e+00
7,41380.4022052600,41380.4022052600,5.175794e+04,3.773280e+04,MPI_Irecv(),9.66692693163508,9.66692693163508,3.716963e+01
8,41380.4022052600,41380.4022052600,5.175794e+04,3.773280e+04,MPI_Isend(),9.66692693163508,9.66692693163508,3.716963e+01
9,0.999999999999999,0.999999999999999,1.000000e+00,1.000000e+00,MPI_Reduce(),1.11022302462516e-13,1.11022302462516e-13,0.000000e+00


In [7]:
print(DF_about_call_at_lulesh.to_csv())

,multi-parameter,"default --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1",proposal,real_call,functionName,real_call vs default,real_call vs multi-parameter,real_call vs proposal
0,0.999999999999999,0.999999999999999,1.0,1.0,.TAU_application,1.11022302462516e-13,1.11022302462516e-13,0.0
1,1023.00000000000,1023.00000000000,1023.0,1023.0,MPI_Allreduce(),2.22261657324763e-14,2.22261657324763e-14,0.0
2,0.999999999999999,0.999999999999999,1.0,1.0,MPI_Barrier(),1.11022302462516e-13,1.11022302462516e-13,0.0
3,9221.00000000000,9221.00000000000,9221.000000000004,9221.0,MPI_Comm_rank(),1.97265958523572e-14,1.97265958523572e-14,3.945319170471438e-14
4,0.999999999999999,0.999999999999999,1.0,1.0,MPI_Comm_size(),1.11022302462516e-13,1.11022302462516e-13,0.0
5,0.999999999999999,0.999999999999999,1.0,1.0,MPI_Finalize(),1.11022302462516e-13,1.11022302462516e-13,0.0
6,0.999999999999999,0.999999999999999,1.0,1.0,MPI_Init(),1.11022302462516e-

In [8]:
DF_about_call_at_lulesh.mean()

/tmp/ipykernel_26507/1662607098.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  DF_about_call_at_lulesh.mean()


multi-parameter                                                                                                           6.717916e+07
default --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1    6.717916e+07
proposal                                                                                                                  6.481530e+07
real_call                                                                                                                 6.717865e+07
real_call vs default                                                                                                      9.840997e+01
real_call vs multi-parameter                                                                                              9.840997e+01
real_call vs proposal                                                                                                     3.594678e+00
dtype: float64

In [9]:
# CG

In [10]:
trainDF_CG :pd.DataFrame = return_rawDF_cg(
    list_process=train_npb_process,
    list_na = train_cg_na,
    list_nonzer=train_cg_nonzer,
    list_niter=train_cg_niter,
    list_shift=train_cg_shift,
    csvDir=csvDirPath
)

testDF_CG :pd.DataFrame = return_rawDF_cg(
    list_process=test_npb_process,
    list_na = test_cg_na,
    list_nonzer=test_cg_nonzer,
    list_niter=test_cg_niter,
    list_shift=test_cg_shift,
    csvDir=csvDirPath
)

trainDF_CG = trainDF_CG.reset_index()
testDF_CG = testDF_CG.reset_index()

trainDF_CG["functionName"] = trainDF_CG["Name"]
testDF_CG["functionName"] = testDF_CG["Name"]

functionNames_CG :list[str] = sorted(list(set(trainDF_CG["Name"])))

expVar_CG :list[str] = ["process", "na", "nonzer", "niter", "shift"]
resVar_CG :str = "#Call"

dict_symbols_CG = {}
for elem in expVar_CG:
    dict_symbols_CG[elem] = symbols(elem, real=True)

target_env_cg = [
    (dict_symbols_CG["process"], test_npb_process[-1]),
    (dict_symbols_CG["na"], test_cg_na[-1]),
    (dict_symbols_CG["nonzer"], test_cg_nonzer[-1]),
    (dict_symbols_CG["niter"], test_cg_niter[-1]),
    (dict_symbols_CG["shift"], test_cg_shift[-1])
]

/workspace/lib/lab_lib.py:8785: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/workspace/lib/lab_lib.py:8785: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [11]:
# 関数コール回数予測

dict_models_perFunc_CG :dict[str, dict[str, str]] = []

for functionName in functionNames_CG:
    pass

trainDF_perFunc :pd.DataFrame = trainDF_CG[trainDF_CG["Name"] == functionName].reset_index()

str_ExtraPinputData_call :str = gen_ExtraPinputDataFromDF(
    inputDF = trainDF_perFunc,
    expVar = expVar_CG,
    resVar = resVar_CG,
)

# モデル格納用の辞書
dictModels :dict[str, str] = {}

# Extra-Pによるモデルの構築
converted_functionName :str = re.sub(r'[\\|/|:|?|.|"|<|>|\|\(|\)|]', "-", functionName)
fileName :str = f"input_cg@{converted_functionName}"
fileDir :str = f"./extra-p_docker/share/"
filePath:str = fileDir + fileName

with open(file=filePath, mode="w") as f:
    f.write(str_ExtraPinputData_call)

for modelerName in modelerNames:
    str_resFromExtraP :str = subprocess.run(
        f"extrap --text {filePath} --modeler {modelerName} --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1 | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout
    str_resFromExtraP = str_resFromExtraP.replace("Model: ", "")
    str_resFromExtraP = convert_log(str_resFromExtraP)
    print(str_resFromExtraP)
    model_fromExtraP = sympify(str_resFromExtraP, locals=dict_symbols_CG)
    dictModels[modelerName]

# 提案手法によるモデルの構築
bestModelDict:dict = return_bestModelObject(
    inputDF = trainDF_perFunc,
    list_expVar=expVar_CG,
    list_resVar = [resVar_CG],
    list_modelName=list_modelName,
)

dictModels["proposal"] = bestModelDict["object"]

dict_models_perFunc_CG[functionName] = dictModels

usage: --modeler multi-parameter --options
                                           [single_parameter_point_selection ['auto', 'smallest', 'all']]
                                           [allow_combinations_of_sums_and_products BOOL]
                                           [compare_with_RSS BOOL]
                                           [#spm {Basic,Refining,Default}]
                                           [#spo KEY=VALUE [KEY=VALUE ...]]
--modeler multi-parameter --options: error: unrecognized arguments: poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1


SympifyError: Sympify of expression 'could not parse ''' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 0)

In [ ]:
_list_series :list[pd.Series] = []

for functionNames in functionNames_CG:
    pass


-----

=====


In [ ]:
def ret_averaged_rawDF_cg(
    list_process=list[int],
    list_na=list[int],
    list_nonzer=list[int],
    list_niter=list[int],
    list_shift=list[int],
    list_csvDir=list[str],
    resVar=str,
):
    """複数のCSVからDFを取得する関数

    引数resVarで指定された列がInclusiveもしくはExclusiveの場合はそれらが秒に変換され、InclusivePerCall列もしくはExclusivePerCall列が生成される

    Args:
        list_process(list[int]):プロセス数
        list_na(list[int]):初期変数na
        list_nonzer(list[int]):初期変数nonzer
        list_niter(list[int]):初期変数niter
        list_shift(list[int]):初期変数shift
        list_csvDir(list[str]):CSVファイルのディレクトリを複数保持したリスト
        resVar(str):目的変数のカラム名
    """
    pass